In [2]:
# From Base (Researcher)

import datetime
import numpy as np
from pathlib import Path
import json
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from mcsim.analysis import fit_dmd_affine, mm_io
import mcsim.analysis.dmd_patterns as dmd_pats
from mcsim.expt_ctrl import dlp6500


In [2]:
# Upload DMD Calibration Mask

dmd_size = [1920, 1080]
masks, radii, pattern_centers = dmd_pats.get_affine_fit_pattern(dmd_size)
mask = masks[1]
mask = mask.astype('uint8')

# Connect to DMD
dmd = dlp6500.dlp6500win(debug=False) 

# DMD Upoload Function
def Upload_Arr_to_DMD(arr,dmd):
    pat = arr
    exposure_t = 150
    dark_t = 0
    triggered = False
    img_inds, bit_inds = dmd.upload_pattern_sequence(pat, exposure_t, dark_t, triggered, clear_pattern_after_trigger = False, bit_depth = 1, num_repeats = 0)
    dmd.set_pattern_sequence(img_inds, bit_inds, exposure_t, dark_t, triggered=False, clear_pattern_after_trigger= False, bit_depth=1, num_repeats=0, mode='on-the-fly')
    dmd.start_stop_sequence("start")
    
    
    
    
mask_Arr = np.ones((1080,1920), np.uint8)
mask_Arr[~mask]=0
Upload_Arr_to_DMD(mask,dmd)

In [25]:
import mcsim.analysis.dmd_patterns as dmd
from matplotlib.patches import Rectangle
from localize_psf import affine, rois, fit

# atlas10 width = 67.07um
# atlas20 height = 56.12 um
# Pixel pitch - 2.74 um 


# ###########################
# set image data location
# ###########################

# CALIBRATION IMAGE FILE NAME!!!! CHANGE EVERY TIME!!!
img_fname = Path("data", "C3.tif")
channel_labels = ["red"]
time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H;%M;%S")
save_dir = img_fname.parent / f"{time_stamp:s}_affine_calibration"


# ###########################
# set guesses for three "spots" from DMD pattern
# ###########################


#centers_init = [[1039, 918], [982, 976], [1091, 979]]
#indices_init = [[10, 16], [9, 16], [10, 15]] #[dmd short axis, dmd long axis]



#centers_init = [[1054,1328,], [880,1328],  [882,1156] ] # [Y,X] BACKWARDS! WORKED WITH 12_16 !
indices_init = [[8, 16], [9, 16], [9,15]] #[dmd short axis, dmd long axis] WORKED WITH 12_16 !

#centers_init = [[1056,1454,], [884,1458],  [886,1286]]
centers_init = [[1267,1060], [1267,1083],  [1290,1084]]
#centers_init = [[1060,1267], [1083,1267],  [1084, 1290]]


cents = np.array(centers_init)



# ###########################
# set other parameters for fitting
# ###########################

#roi_size = 100 #  May need to change this 
roi_size = 10 #  May need to change this



options = {'cam_pix': 1.4e-6,
           #'cam_pix': 1.4e-6*5,
           #'dmd_pix': 7.56e-6,
           'dmd_pix': 0.5e-6, # 897p for 450um
           'dmd2cam_mag_expected': 1,
           #'dmd2cam_mag_expected': 180 / 300 * 400 / 200,
           #'cam_mag': 100
           'cam_mag': 1
             }

def sigma_pix(wl1, wl2, na, cam_mag):
    print("Hello")
    return np.sqrt(wl1**2 + wl2**2) / 2 / na / (2 * np.sqrt(2 * np.log(2))) / (options["cam_pix"] / cam_mag)

# load DMD pattern and dmd_centers
dmd_size = [1920, 1080]
masks, radii, pattern_centers = dmd.get_affine_fit_pattern(dmd_size)
mask = masks[1]


# ###########################
# perform affine calibration for each channel and plot/export results
# ###########################
affine_summary = {}

for nc in range(len(channel_labels)):
    img, _ = mm_io.read_tiff(img_fname, slices=nc)
    #img, _ = mm_io.read_tiff(img_fname)
    ii = img 
    
    img_T = np.transpose(img,(2,0,1))
    
   

    

    #img = img[0]
    img = img_T[0]
    img = ii[0,:,:]
    
    fig = plt.figure()
    ax = fig.add_subplot(1,2,1)
    ax.imshow(img)
    ax.plot(cents[:,1],cents[:,0], 'rx')
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(mask)
    dmd_cents_plot = np.array([pattern_centers[i[0],i[1]] for i in indices_init])
    
    ax2.plot(dmd_cents_plot[:,0], dmd_cents_plot[:,1], 'rx')
    plt.show()
    
    
    
    
    #img = np.uint16(img)
    #img = np.flipud(img)
    
    affine_xform_data, figh = fit_dmd_affine.estimate_xform(img,
                                                            mask,
                                                            pattern_centers,
                                                            centers_init,
                                                            indices_init,
                                                            options,
                                                            roi_size=roi_size,
                                                            export_fname=f"affine_xform_{channel_labels[nc]:s}",
                                                            export_dir=save_dir,
                                                            chi_squared_relative_max=3,
                                                            figsize=(20, 12))

    affine_summary[channel_labels[nc]] = affine_xform_data["affine_xform"]

# save summary results
affine_summary["transform_direction"] = "dmd to camera"
affine_summary["processing_time_stamp"] = time_stamp
affine_summary["data_time_stamp"] = ""
affine_summary["file_name"] = str(img_fname)

fname_summary = save_dir / "dmd_affine_transformations.json"
with open(fname_summary, "w") as f:
    json.dump(affine_summary, f, indent="\t")

plt.show()








In [1]:
# Convert Camera Targets to DMD Mask

import datetime
import numpy as np
from pathlib import Path
import json
import matplotlib
#matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from mcsim.analysis import fit_dmd_affine, mm_io
#import mcsim.analysis.dmd_patterns as dmd
from matplotlib.patches import Rectangle
from localize_psf import affine, rois, fit
import cv2

from mcsim.expt_ctrl import dlp6500

global start_Pos, stop_Pos, flag, ix, iy, drawing, display_img, fx, fy

ix = 0
iy = 0
#fx = 2448
#fy = 2048

fx = 2304
fy = 2304

drawing = False
window_width = 2000
window_height = 1300

TAB1 = "  "
TAB2 = "    "
DMD_Width = 1920
DMD_Height = 1080
#Camera_Width = 2448 
#Camera_Height = 2048 
Camera_Width = 2304
Camera_Height = 2304

# Get Transform saved with calibration - DO THIS EVERY TIME BEFORE AQUISITION W/ SLIDE!
img_fname = Path("data", "C3.tif")
save_dir = img_fname.parent /"2024_10_16_13;32;31_affine_calibration"
fname_sum = save_dir / "affine_xform_red.json"
with open(fname_sum) as f:
   dmd_affine_transformations = json.load(f)

affine_xform_list = list(dmd_affine_transformations.values())
affine_xform_values = affine_xform_list[0]

affine_xform = np.ones((3,3))
affine_xform[0] = affine_xform_values[0]
affine_xform[1] = affine_xform_values[1]
affine_xform[2] = affine_xform_values[2]

affine_xform_inv = np.linalg.inv(affine_xform)

# Make Image 
img = np.ones((Camera_Height, Camera_Width),np.uint8)
img_coords = np.meshgrid(range(img.shape[1]), range(img.shape[0]))



dmd_all_on = np.ones((DMD_Height, DMD_Width),np.uint8)
dmd_coords = np.meshgrid(range(dmd_all_on.shape[1]), range(dmd_all_on.shape[0]))

# Convert to DMD Space
dmd_pattern = affine.xform_mat(img, affine_xform_inv, dmd_coords, mode='nearest')
dmd_pattern[np.isnan(dmd_pattern)] = 0
dmd_pattern = dmd_pattern.astype('uint8')

dmd = dlp6500.dlp6500win(debug=False) 

# DMD Upoload Function
def Upload_Arr_to_DMD(arr,dmd):
    pat = arr
    exposure_t = 150
    dark_t = 0
    triggered = False
    img_inds, bit_inds = dmd.upload_pattern_sequence(pat, exposure_t, dark_t, triggered, clear_pattern_after_trigger = False, bit_depth = 1, num_repeats = 0)
    dmd.set_pattern_sequence(img_inds, bit_inds, exposure_t, dark_t, triggered=False, clear_pattern_after_trigger= False, bit_depth=1, num_repeats=0, mode='on-the-fly')
    dmd.start_stop_sequence("start")
    

def draw_reactangle_with_drag(event, x, y, flags, param):
    
    # ix is starting x coord, it is easiest to start left and move right 
    # iy is the starting y coord, this goes top to bottem 
    # fx is the final x coord, this should be the right most 
    # fy is the final y coord, this should toward bottem 
    
    global ix, iy, drawing, display_img, fx, fy
    #print("Draw Time")
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix = x
        iy = y
        print(iy)
       
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            img2 = cv2.imread("data//S1.jpg") # Change this to full FOV frame taken of real sample!!!
            cv2.rectangle(img2, pt1=(ix,iy), pt2=(x, y),color=(255,255,255),thickness=10)
            display_img = img2

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        img2 = cv2.imread("data//S1.jpg") # Change this to full FOV frame taken of real sample!!!
        cv2.rectangle(img2, pt1=(ix,iy), pt2=(x, y),color=(255,255,255),thickness=10)
        display_img = img2
        fx = x
        fy = y  


test_path = "data//S1.jpg" # Change this to full FOV frame taken of real sample!!!
key = -1
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.setMouseCallback("Image", draw_reactangle_with_drag)
display_img = cv2.imread(test_path)
cv2.resizeWindow("Image", window_width, window_height)

# Display Until Esc key hit!
while True:
    cv2.imshow("Image", display_img)
    if cv2.waitKey(10) == 27:
        break

print("The box drawn has corners (cam dimensions) at: ",ix,iy,fx,fy)
cv2.destroyAllWindows()

# Use box to make MASK
camera_mask = np.ones((Camera_Height, Camera_Width), np.uint8)

for i in range(Camera_Width):
        if i < ix:
            camera_mask[:,i] = 0
        elif i > fx:
            camera_mask[:,i] = 0

for j in range(Camera_Height):
        if j < iy:
            camera_mask[j,:] = 0
        elif j > fy:
            camera_mask[j,:] = 0
            
            

dmd_pattern = affine.xform_mat(camera_mask, affine_xform_inv, dmd_coords, mode='nearest')
dmd_pattern[np.isnan(dmd_pattern)] = 0
dmd_pattern = dmd_pattern.astype('uint8')
Upload_Arr_to_DMD(dmd_pattern,dmd)



1189
The box drawn has corners (cam dimensions) at:  985 1189 1031 1276
